In [1]:
import pandas as pd
import numpy as np
import math
import xlsxwriter
import requests
from scipy import stats

In [2]:
stocks= pd.read_csv('sp_500_stocks.csv')

In [3]:
from kamka import IEX_CLOUD_API_TOKEN

In [4]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]  
        
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_cols= ['Ticker', 'Price', 'One year price return', 'Number of shares to buy']
    

In [5]:
final_dataframe = pd.DataFrame(columns= my_cols)
final_dataframe

,Ticker,Price,One year price return,Number of shares to buy


In [6]:
for symbol_string in symbol_strings:
    batch_url= f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_url).json()
    for symbol in symbol_string.split(','):
        oypr= data[symbol]['stats']['year1ChangePercent']
        price= data[symbol]['quote']['latestPrice']
        final_dataframe.loc[len(final_dataframe)] = [symbol, price,oypr, 'NA']

final_dataframe

C:\Users\deepa\AppData\Local\Temp\ipykernel_26640\1280888331.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe.loc[len(final_dataframe)] = [symbol, price,oypr, 'NA']
C:\Users\deepa\AppData\Local\Temp\ipykernel_26640\1280888331.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe.loc[len(final_dataframe)] = [symbol, price,oypr, 'NA']
C:\Users\deepa\AppData\Local\Temp\ipykernel_26640\1280888331.py:7: FutureWarning: The behavior of DataFrame concatenation with empty

,Ticker,Price,One year price return,Number of shares to buy
0,A,127.20,-0.180326,NA
1,AAL,13.76,0.011765,NA
2,AAP,56.27,-0.603904,NA
3,AAPL,195.71,0.385056,NA
4,ABBV,149.28,-0.064337,NA
...,...,...,...,...
486,YUM,124.34,-0.024416,NA
487,ZBH,117.30,-0.060723,NA
488,ZBRA,236.13,-0.066015,NA
489,ZION,38.93,-0.137211,NA


In [7]:
final_dataframe.sort_values('One year price return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One year price return,Number of shares to buy
0,LB,79.92,2.281287,NA
1,NVDA,475.06,1.768191,NA
2,PHM,96.31,1.191863,NA
3,RCL,120.47,1.110177,NA
4,CCL,18.11,1.005537,NA
5,CRM,250.81,0.927380,NA
6,GE,120.59,0.838335,NA
7,AVGO,944.30,0.835050,NA
8,ADBE,610.01,0.834175,NA
9,AMD,128.92,0.829431,NA


In [8]:
hqm_columns = ['Ticker', 
                'Price',  
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score','Number of Shares to Buy', ]

hqm_dataframe= pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_url= f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data= requests.get(batch_url).json()
    for symbol in symbol_string.split(','):
        oney= data[symbol]['stats']['year1ChangePercent']
        onem = data[symbol]['stats']['month1ChangePercent']
        threem = data[symbol]['stats']['month3ChangePercent']
        sixm = data[symbol]['stats']['month6ChangePercent']
        hqm_dataframe.loc[len(hqm_dataframe)]=[symbol,data[symbol]['quote']['latestPrice'],oney,'NA',sixm,'NA',threem,'NA',onem,'NA','NA','NA']

hqm_dataframe
        



C:\Users\deepa\AppData\Local\Temp\ipykernel_26640\3650171725.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe.loc[len(hqm_dataframe)]=[symbol,data[symbol]['quote']['latestPrice'],oney,'NA',sixm,'NA',threem,'NA',onem,'NA','NA','NA']
C:\Users\deepa\AppData\Local\Temp\ipykernel_26640\3650171725.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe.loc[len(hqm_dataframe)]=[symbol,data[symbol]['quote']['latestPrice'],oney,'NA',sixm,'NA',threem,'NA',onem,'NA','NA','NA']


,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
0,A,127.20,-0.180326,NA,0.113435,NA,0.118138,NA,0.162812,NA,NA,NA
1,AAL,13.76,0.011765,NA,-0.100654,NA,-0.015737,NA,0.145712,NA,NA,NA
2,AAP,56.27,-0.603904,NA,-0.123474,NA,-0.124814,NA,-0.023768,NA,NA,NA
3,AAPL,195.71,0.385056,NA,0.089640,NA,0.101279,NA,0.072918,NA,NA,NA
4,ABBV,149.28,-0.064337,NA,0.107569,NA,0.011771,NA,0.051046,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
486,YUM,124.34,-0.024416,NA,-0.057707,NA,-0.012660,NA,-0.005429,NA,NA,NA
487,ZBH,117.30,-0.060723,NA,-0.129646,NA,-0.025655,NA,0.118634,NA,NA,NA
488,ZBRA,236.13,-0.066015,NA,-0.150703,NA,-0.078948,NA,0.143819,NA,NA,NA
489,ZION,38.93,-0.137211,NA,0.245469,NA,0.128403,NA,0.153140,NA,NA,NA


In [9]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for time_period in time_periods:
    try:
        hqm_dataframe[f'{time_period} Return Percentile'] = hqm_dataframe[f'{time_period} Price Return'].apply(
            lambda x: stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'].dropna(), x) / 100
            if pd.notna(x) else None
        )
    except Exception as e:
        print(f"Error calculating percentiles for {time_period}: {e}")

hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
0,A,127.20,-0.180326,0.121150,0.113435,0.679671,0.118138,0.798768,0.162812,0.901437,NA,NA
1,AAL,13.76,0.011765,0.466119,-0.100654,0.145791,-0.015737,0.308008,0.145712,0.872690,NA,NA
2,AAP,56.27,-0.603904,0.006160,-0.123474,0.117043,-0.124814,0.082136,-0.023768,0.086242,NA,NA
3,AAPL,195.71,0.385056,0.887064,0.089640,0.611910,0.101279,0.739220,0.072918,0.533881,NA,NA
4,ABBV,149.28,-0.064337,0.277207,0.107569,0.663244,0.011771,0.447639,0.051046,0.400411,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
486,YUM,124.34,-0.024416,0.361396,-0.057707,0.203285,-0.012660,0.328542,-0.005429,0.123203,NA,NA
487,ZBH,117.30,-0.060723,0.279261,-0.129646,0.108830,-0.025655,0.281314,0.118634,0.792608,NA,NA
488,ZBRA,236.13,-0.066015,0.273101,-0.150703,0.102669,-0.078948,0.143737,0.143819,0.868583,NA,NA
489,ZION,38.93,-0.137211,0.158111,0.245469,0.905544,0.128403,0.823409,0.153140,0.882957,NA,NA


In [10]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
0,A,127.20,-0.180326,0.121150,0.113435,0.679671,0.118138,0.798768,0.162812,0.901437,0.625257,NA
1,AAL,13.76,0.011765,0.466119,-0.100654,0.145791,-0.015737,0.308008,0.145712,0.872690,0.448152,NA
2,AAP,56.27,-0.603904,0.006160,-0.123474,0.117043,-0.124814,0.082136,-0.023768,0.086242,0.072895,NA
3,AAPL,195.71,0.385056,0.887064,0.089640,0.611910,0.101279,0.739220,0.072918,0.533881,0.693018,NA
4,ABBV,149.28,-0.064337,0.277207,0.107569,0.663244,0.011771,0.447639,0.051046,0.400411,0.447125,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
486,YUM,124.34,-0.024416,0.361396,-0.057707,0.203285,-0.012660,0.328542,-0.005429,0.123203,0.254107,NA
487,ZBH,117.30,-0.060723,0.279261,-0.129646,0.108830,-0.025655,0.281314,0.118634,0.792608,0.365503,NA
488,ZBRA,236.13,-0.066015,0.273101,-0.150703,0.102669,-0.078948,0.143737,0.143819,0.868583,0.347023,NA
489,ZION,38.93,-0.137211,0.158111,0.245469,0.905544,0.128403,0.823409,0.153140,0.882957,0.692505,NA


In [11]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [ ]:
portfolio_size=input()

position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

In [ ]:
with pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter') as writer:
    
    hqm_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Recommended Trades']
    column_settings = [{'header': column} for column in final_dataframe.columns]
    worksheet.add_table(0, 0, final_dataframe.shape[0], final_dataframe.shape[1] - 1, {'columns': column_settings})